In [1]:
# Sneha Dubey & Hanzhe Zhang
# Dr. Chen
# CSCI 185
# 20 March 2024
# Final Project

In [15]:
# Imports

import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [3]:
odata = pd.read_csv("dataset.csv")
odata.info()
odata.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1998 entries, 0 to 1997
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Job Id            1998 non-null   float64
 1   Experience        1998 non-null   object 
 2   Qualifications    1998 non-null   object 
 3   Salary Range      1998 non-null   object 
 4   location          1998 non-null   object 
 5   Country           1998 non-null   object 
 6   latitude          1998 non-null   float64
 7   longitude         1998 non-null   float64
 8   Work Type         1998 non-null   object 
 9   Company Size      1998 non-null   int64  
 10  Job Posting Date  1998 non-null   object 
 11  Preference        1998 non-null   object 
 12  Contact Person    1998 non-null   object 
 13  Contact           1998 non-null   object 
 14  Job Title         1998 non-null   object 
 15  Role              1998 non-null   object 
 16  Job Portal        1998 non-null   object 


,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Contact,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Company Profile
0,1.089840e+15,5 to 15 Years,M.Tech,$59K-$99K,Douglas,Isle of Man,54.2361,-4.5481,Intern,26801,...,001-381-930-7517x737,Digital Marketing Specialist,Social Media Manager,Snagajob,Social Media Managers oversee an organizations...,"{'Flexible Spending Accounts (FSAs), Relocatio...","Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",Icahn Enterprises,"{""Sector"":""Diversified"",""Industry"":""Diversifie..."
1,3.984540e+14,2 to 12 Years,BCA,$56K-$116K,Ashgabat,Turkmenistan,38.9697,59.5563,Intern,100340,...,461-509-4216,Web Developer,Frontend Web Developer,Idealist,Frontend Web Developers design and implement u...,"{'Health Insurance, Retirement Plans, Paid Tim...","HTML, CSS, JavaScript Frontend frameworks (e.g...","Design and code user interfaces for websites, ...",PNC Financial Services Group,"{""Sector"":""Financial Services"",""Industry"":""Com..."
2,4.816400e+14,0 to 12 Years,PhD,$61K-$104K,Macao,"Macao SAR, China",22.1987,113.5439,Temporary,84525,...,9687619505,Operations Manager,Quality Control Manager,Jobs2Careers,Quality Control Managers establish and enforce...,"{'Legal Assistance, Bonuses and Incentive Prog...",Quality control processes and methodologies St...,Establish and enforce quality control standard...,United Services Automobile Assn.,"{""Sector"":""Insurance"",""Industry"":""Insurance: P..."
3,6.881930e+14,4 to 11 Years,PhD,$65K-$91K,Porto-Novo,Benin,9.3077,2.3158,Full-Time,129896,...,+1-820-643-5431x47576,Network Engineer,Wireless Network Engineer,FlexJobs,"Wireless Network Engineers design, implement, ...","{'Transportation Benefits, Professional Develo...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Hess,"{""Sector"":""Energy"",""Industry"":""Mining, Crude-O..."
4,1.170580e+14,1 to 12 Years,MBA,$64K-$87K,Santiago,Chile,-35.6751,-71.5429,Intern,53944,...,343.975.4702x9340,Event Manager,Conference Manager,Jobs2Careers,A Conference Manager coordinates and manages c...,"{'Flexible Spending Accounts (FSAs), Relocatio...",Event planning Conference logistics Budget man...,Specialize in conference and convention planni...,Cairn Energy,"{""Sector"":""Energy"",""Industry"":""Energy - Oil & ..."


In [4]:
# Begin pre-processing the data (create copy to protect original data)
preprocessedData = odata

In [5]:
# Drop unnecessary columns
preprocessedData = preprocessedData.drop(columns = ['Job Id', 'latitude', 'longitude', 'Contact', 'Job Portal', 'Company Size'], axis = 1)
for column in preprocessedData.columns:
    preprocessedData[column].fillna('nan', inplace = True)
preprocessedData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1998 entries, 0 to 1997
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Experience        1998 non-null   object
 1   Qualifications    1998 non-null   object
 2   Salary Range      1998 non-null   object
 3   location          1998 non-null   object
 4   Country           1998 non-null   object
 5   Work Type         1998 non-null   object
 6   Job Posting Date  1998 non-null   object
 7   Preference        1998 non-null   object
 8   Contact Person    1998 non-null   object
 9   Job Title         1998 non-null   object
 10  Role              1998 non-null   object
 11  Job Description   1998 non-null   object
 12  Benefits          1998 non-null   object
 13  skills            1998 non-null   object
 14  Responsibilities  1998 non-null   object
 15  Company           1998 non-null   object
 16  Company Profile   1998 non-null   object
dtypes: object(17)


C:\Users\zhang\AppData\Local\Temp\ipykernel_24528\497445550.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  preprocessedData[column].fillna('nan', inplace = True)


In [14]:
# Tokenize the data

tokenizedData = preprocessedData
for row in range(0,len(tokenizedData.axes[0])):
    for column in tokenizedData:
        tokenizedData.at[row, column] = str(tokenizedData.at[row, column])

for row in range(0,len(tokenizedData.axes[0])):
    for column in tokenizedData:
        tokenizedData.at[row, column] = word_tokenize(tokenizedData.at[row, column])
        #print(data)

tokenizedData

,Experience,Qualifications,Salary Range,location,Country,Work Type,Job Posting Date,Preference,Contact Person,Job Title,Role,Job Description,Benefits,skills,Responsibilities,Company,Company Profile
0,"[[, ', 5, ', ,, 'to, ', ,, '15, ', ,, 'Years, ...","[[, 'M.Tech, ', ]]","[[, ', $, ', ,, '59K-, ', ,, ', $, ', ,, '99K,...","[[, 'Douglas, ', ]]","[[, 'Isle, ', ,, 'of, ', ,, 'Man, ', ]]","[[, 'Intern, ', ]]","[[, ', 4/24/2022, ', ]]","[[, 'Female, ', ]]","[[, 'Brandon, ', ,, 'Cunningham, ', ]]","[[, 'Digital, ', ,, 'Marketing, ', ,, 'Special...","[[, 'Social, ', ,, 'Media, ', ,, 'Manager, ', ]]","[[, 'Social, ', ,, 'Media, ', ,, 'Managers, ',...","[[, ', {, ', ,, ``, 'Flexible, '', ,, 'Spendin...","[[, 'Social, ', ,, 'media, ', ,, 'platforms, '...","[[, 'Manage, ', ,, 'and, ', ,, 'grow, ', ,, 's...","[[, 'Icahn, ', ,, 'Enterprises, ', ]]","[[, ', {, ', ,, ', ``, ', ,, 'Sector, ', ,, ``..."
1,"[[, ', 2, ', ,, 'to, ', ,, '12, ', ,, 'Years, ...","[[, 'BCA, ', ]]","[[, ', $, ', ,, '56K-, ', ,, ', $, ', ,, '116K...","[[, 'Ashgabat, ', ]]","[[, 'Turkmenistan, ', ]]","[[, 'Intern, ', ]]","[[, '12/19/2022, ', ]]","[[, 'Female, ', ]]","[[, 'Francisco, ', ,, 'Larsen, ', ]]","[[, 'Web, ', ,, 'Developer, ', ]]","[[, 'Frontend, ', ,, 'Web, ', ,, 'Developer, '...","[[, 'Frontend, ', ,, 'Web, ', ,, 'Developers, ...","[[, ', {, ', ,, ``, 'Health, '', ,, 'Insurance...","[[, 'HTML, ', ,, ', ,, ', ,, 'CSS, ', ,, ', ,,...","[[, 'Design, ', ,, 'and, ', ,, 'code, ', ,, 'u...","[[, 'PNC, ', ,, 'Financial, ', ,, 'Services, '...","[[, ', {, ', ,, ', ``, ', ,, 'Sector, ', ,, ``..."
2,"[[, ', 0, ', ,, 'to, ', ,, '12, ', ,, 'Years, ...","[[, 'PhD, ', ]]","[[, ', $, ', ,, '61K-, ', ,, ', $, ', ,, '104K...","[[, 'Macao, ', ]]","[[, 'Macao, ', ,, 'SAR, ', ,, ', ,, ', ,, 'Chi...","[[, 'Temporary, ', ]]","[[, ', 9/14/2022, ', ]]","[[, 'Male, ', ]]","[[, 'Gary, ', ,, 'Gibson, ', ]]","[[, 'Operations, ', ,, 'Manager, ', ]]","[[, 'Quality, ', ,, 'Control, ', ,, 'Manager, ...","[[, 'Quality, ', ,, 'Control, ', ,, 'Managers,...","[[, ', {, ', ,, ``, 'Legal, '', ,, 'Assistance...","[[, 'Quality, ', ,, 'control, ', ,, 'processes...","[[, 'Establish, ', ,, 'and, ', ,, 'enforce, ',...","[[, 'United, ', ,, 'Services, ', ,, 'Automobil...","[[, ', {, ', ,, ', ``, ', ,, 'Sector, ', ,, ``..."
3,"[[, ', 4, ', ,, 'to, ', ,, '11, ', ,, 'Years, ...","[[, 'PhD, ', ]]","[[, ', $, ', ,, '65K-, ', ,, ', $, ', ,, '91K,...","[[, 'Porto-Novo, ', ]]","[[, 'Benin, ', ]]","[[, 'Full-Time, ', ]]","[[, ', 2/25/2023, ', ]]","[[, 'Female, ', ]]","[[, 'Joy, ', ,, 'Lucero, ', ]]","[[, 'Network, ', ,, 'Engineer, ', ]]","[[, 'Wireless, ', ,, 'Network, ', ,, 'Engineer...","[[, 'Wireless, ', ,, 'Network, ', ,, 'Engineer...","[[, ', {, ', ,, ``, 'Transportation, '', ,, 'B...","[[, 'Wireless, ', ,, 'network, ', ,, 'design, ...","[[, 'Design, ', ,, ', ,, ', ,, 'configure, ', ...","[[, 'Hess, ', ]]","[[, ', {, ', ,, ', ``, ', ,, 'Sector, ', ,, ``..."
4,"[[, ', 1, ', ,, 'to, ', ,, '12, ', ,, 'Years, ...","[[, 'MBA, ', ]]","[[, ', $, ', ,, '64K-, ', ,, ', $, ', ,, '87K,...","[[, 'Santiago, ', ]]","[[, 'Chile, ', ]]","[[, 'Intern, ', ]]","[[, '10/11/2022, ', ]]","[[, 'Female, ', ]]","[[, 'Julie, ', ,, 'Johnson, ', ]]","[[, 'Event, ', ,, 'Manager, ', ]]","[[, 'Conference, ', ,, 'Manager, ', ]]","[[, ', A, ', ,, 'Conference, ', ,, 'Manager, '...","[[, ', {, ', ,, ``, 'Flexible, '', ,, 'Spendin...","[[, 'Event, ', ,, 'planning, ', ,, 'Conference...","[[, 'Specialize, ', ,, 'in, ', ,, 'conference,...","[[, 'Cairn, ', ,, 'Energy, ', ]]","[[, ', {, ', ,, ', ``, ', ,, 'Sector, ', ,, ``..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993,"[[, ', 2, ', ,, 'to, ', ,, '10, ', ,, 'Years, ...","[[, 'M.Tech, ', ]]","[[, ', $, ', ,, '63K-, ', ,, ', $, ', ,, '111K...","[[, 'Juba, ', ]]","[[, 'South, ', ,, 'Sudan, ', ]]","[[, 'Contract, ', ]]","[[, ', 3/27/2023, ', ]]","[[, 'Male, ', ]]","[[, 'Kylie, ', ,, 'Copeland, ', ]]","[[, 'Data, ', ,, 'Engineer, ', ]]","[[, 'Big, ', ,, 'Data, ', ,, 'Eng

In [18]:
# Removing stop words
stopWords = set(stopwords.words('english'))
                
for row in range(0,len(tokenizedData.axes[0])):
    for column in tokenizedData:
        assessedList = []
        for w in tokenizedData.at[row, column]:
                if w not in stopWords:
                    assessedList.append(w)
        tokenizedData.at[row, column] = assessedList

tokenizedData

,Experience,Qualifications,Salary Range,location,Country,Work Type,Job Posting Date,Preference,Contact Person,Job Title,Role,Job Description,Benefits,skills,Responsibilities,Company,Company Profile
0,"[[, ', 5, ', ,, 'to, ', ,, '15, ', ,, 'Years, ...","[[, 'M.Tech, ', ]]","[[, ', $, ', ,, '59K-, ', ,, ', $, ', ,, '99K,...","[[, 'Douglas, ', ]]","[[, 'Isle, ', ,, 'of, ', ,, 'Man, ', ]]","[[, 'Intern, ', ]]","[[, ', 4/24/2022, ', ]]","[[, 'Female, ', ]]","[[, 'Brandon, ', ,, 'Cunningham, ', ]]","[[, 'Digital, ', ,, 'Marketing, ', ,, 'Special...","[[, 'Social, ', ,, 'Media, ', ,, 'Manager, ', ]]","[[, 'Social, ', ,, 'Media, ', ,, 'Managers, ',...","[[, ', {, ', ,, ``, 'Flexible, '', ,, 'Spendin...","[[, 'Social, ', ,, 'media, ', ,, 'platforms, '...","[[, 'Manage, ', ,, 'and, ', ,, 'grow, ', ,, 's...","[[, 'Icahn, ', ,, 'Enterprises, ', ]]","[[, ', {, ', ,, ', ``, ', ,, 'Sector, ', ,, ``..."
1,"[[, ', 2, ', ,, 'to, ', ,, '12, ', ,, 'Years, ...","[[, 'BCA, ', ]]","[[, ', $, ', ,, '56K-, ', ,, ', $, ', ,, '116K...","[[, 'Ashgabat, ', ]]","[[, 'Turkmenistan, ', ]]","[[, 'Intern, ', ]]","[[, '12/19/2022, ', ]]","[[, 'Female, ', ]]","[[, 'Francisco, ', ,, 'Larsen, ', ]]","[[, 'Web, ', ,, 'Developer, ', ]]","[[, 'Frontend, ', ,, 'Web, ', ,, 'Developer, '...","[[, 'Frontend, ', ,, 'Web, ', ,, 'Developers, ...","[[, ', {, ', ,, ``, 'Health, '', ,, 'Insurance...","[[, 'HTML, ', ,, ', ,, ', ,, 'CSS, ', ,, ', ,,...","[[, 'Design, ', ,, 'and, ', ,, 'code, ', ,, 'u...","[[, 'PNC, ', ,, 'Financial, ', ,, 'Services, '...","[[, ', {, ', ,, ', ``, ', ,, 'Sector, ', ,, ``..."
2,"[[, ', 0, ', ,, 'to, ', ,, '12, ', ,, 'Years, ...","[[, 'PhD, ', ]]","[[, ', $, ', ,, '61K-, ', ,, ', $, ', ,, '104K...","[[, 'Macao, ', ]]","[[, 'Macao, ', ,, 'SAR, ', ,, ', ,, ', ,, 'Chi...","[[, 'Temporary, ', ]]","[[, ', 9/14/2022, ', ]]","[[, 'Male, ', ]]","[[, 'Gary, ', ,, 'Gibson, ', ]]","[[, 'Operations, ', ,, 'Manager, ', ]]","[[, 'Quality, ', ,, 'Control, ', ,, 'Manager, ...","[[, 'Quality, ', ,, 'Control, ', ,, 'Managers,...","[[, ', {, ', ,, ``, 'Legal, '', ,, 'Assistance...","[[, 'Quality, ', ,, 'control, ', ,, 'processes...","[[, 'Establish, ', ,, 'and, ', ,, 'enforce, ',...","[[, 'United, ', ,, 'Services, ', ,, 'Automobil...","[[, ', {, ', ,, ', ``, ', ,, 'Sector, ', ,, ``..."
3,"[[, ', 4, ', ,, 'to, ', ,, '11, ', ,, 'Years, ...","[[, 'PhD, ', ]]","[[, ', $, ', ,, '65K-, ', ,, ', $, ', ,, '91K,...","[[, 'Porto-Novo, ', ]]","[[, 'Benin, ', ]]","[[, 'Full-Time, ', ]]","[[, ', 2/25/2023, ', ]]","[[, 'Female, ', ]]","[[, 'Joy, ', ,, 'Lucero, ', ]]","[[, 'Network, ', ,, 'Engineer, ', ]]","[[, 'Wireless, ', ,, 'Network, ', ,, 'Engineer...","[[, 'Wireless, ', ,, 'Network, ', ,, 'Engineer...","[[, ', {, ', ,, ``, 'Transportation, '', ,, 'B...","[[, 'Wireless, ', ,, 'network, ', ,, 'design, ...","[[, 'Design, ', ,, ', ,, ', ,, 'configure, ', ...","[[, 'Hess, ', ]]","[[, ', {, ', ,, ', ``, ', ,, 'Sector, ', ,, ``..."
4,"[[, ', 1, ', ,, 'to, ', ,, '12, ', ,, 'Years, ...","[[, 'MBA, ', ]]","[[, ', $, ', ,, '64K-, ', ,, ', $, ', ,, '87K,...","[[, 'Santiago, ', ]]","[[, 'Chile, ', ]]","[[, 'Intern, ', ]]","[[, '10/11/2022, ', ]]","[[, 'Female, ', ]]","[[, 'Julie, ', ,, 'Johnson, ', ]]","[[, 'Event, ', ,, 'Manager, ', ]]","[[, 'Conference, ', ,, 'Manager, ', ]]","[[, ', A, ', ,, 'Conference, ', ,, 'Manager, '...","[[, ', {, ', ,, ``, 'Flexible, '', ,, 'Spendin...","[[, 'Event, ', ,, 'planning, ', ,, 'Conference...","[[, 'Specialize, ', ,, 'in, ', ,, 'conference,...","[[, 'Cairn, ', ,, 'Energy, ', ]]","[[, ', {, ', ,, ', ``, ', ,, 'Sector, ', ,, ``..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993,"[[, ', 2, ', ,, 'to, ', ,, '10, ', ,, 'Years, ...","[[, 'M.Tech, ', ]]","[[, ', $, ', ,, '63K-, ', ,, ', $, ', ,, '111K...","[[, 'Juba, ', ]]","[[, 'South, ', ,, 'Sudan, ', ]]","[[, 'Contract, ', ]]","[[, ', 3/27/2023, ', ]]","[[, 'Male, ', ]]","[[, 'Kylie, ', ,, 'Copeland, ', ]]","[[, 'Data, ', ,, 'Engineer, ', ]]","[[, 'Big, ', ,, 'Data, ', ,, 'Eng

In [20]:
# Beginning Vector Space Model
documents = []
for row in range(0,len(tokenizedData.axes[0])):
    currDoc = []
    for column in tokenizedData:
        for token in tokenizedData.at[row, column]:
            currDoc.append(token)
    documents.append(currDoc)
print(documents)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

